## **1. Przygotowanie zbioru danych**

### **Uwaga!**
Ze względu na poufność danych, surowe raporty i adnotacje ekspertów nie są zawarte w tym repozytorium.

### **Problem:** 
Wieloetykietowa klasyfikacja długich, nieustrukturyzowanych dokumentów w języku polskim, w warunkach silnego niezbalansowania klas.

### **Model enkodera:** sdadas/polish-longformer-base-4096 (https://huggingface.co/sdadas/polish-longformer-base-4096)

#### **Import Bibliotek**

In [1]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
from skmultilearn.model_selection import iterative_train_test_split

In [2]:
with open('data/data.jsonl', 'r') as f:
    df = pd.read_json(f, lines=True)

#### **1.2. Stratyfikacja iteracyjna na głównym zbiorze.**

In [3]:
X = df['text'].values.reshape(-1, 1)
y = np.array(df['labels'].tolist())

In [4]:
# Pierwszy podział: na zbiór treningowy (70%) i tymczasowy (30%)
X_train, y_train, X_temp, y_temp = iterative_train_test_split(
    X, y, test_size=0.3
)

# Drugi podział: dzielimy zbiór tymczasowy (30%) na pół (15% walidacyjny i 15% testowy)
X_val, y_val, X_test, y_test = iterative_train_test_split(
    X_temp, y_temp, test_size=0.5
)

#### **1.3. Stworzenie DataFrame dla zbioru treningowego, testowego i walidacyjnego.**

In [5]:
train_df = pd.DataFrame({
    'text': X_train.ravel(),
    'labels': [list(label) for label in y_train]
})

val_df = pd.DataFrame({
    'text': X_val.ravel(),
    'labels': [list(label) for label in y_val]
})

test_df = pd.DataFrame({
    'text': X_test.ravel(),
    'labels': [list(label) for label in y_test]
})

In [6]:
print(f"Zbiór treningowy: {len(train_df)} raportów")
print(f"Zbiór walidacyjny: {len(val_df)} raportów")
print(f"Zbiór testowy:    {len(test_df)} raportów")

Zbiór treningowy: 275 raportów
Zbiór walidacyjny: 59 raportów
Zbiór testowy:    59 raportów


#### **1.4. Dodanie unikalnych identyfikatorów (doc_id).**

In [7]:
train_df['doc_id'] = [f'train_{i}' for i in range(len(train_df))]
val_df['doc_id'] = [f'val_{i}' for i in range(len(val_df))]
test_df['doc_id'] = [f'test_{i}' for i in range(len(test_df))]

#### **1.5. Wizualizacja podzielonych zbiorów.**

In [8]:
datasets = {
    "Oryginalny": y,
    "Treningowy": y_train,
    "Walidacyjny": y_val,
    "Testowy": y_test
}

plot_data = []
kryteria_names = [f"Kryterium {i+1}" for i in range(y.shape[1])]

for name, labels_array in datasets.items():
    distribution = np.sum(labels_array, axis=0) / len(labels_array)
    for i, percentage in enumerate(distribution):
        plot_data.append({
            "Zbiór Danych": name,
            "Kryterium": kryteria_names[i],
            "Odsetek Wystąpień": percentage
        })

dist_df = pd.DataFrame(plot_data)

In [9]:
fig = px.bar(
    dist_df,
    x="Kryterium",
    y="Odsetek Wystąpień",
    color="Zbiór Danych",
    barmode="group",
    labels={
        "Odsetek Wystąpień": "Odsetek Wystąpień Etykiety '1'",
        "Kryterium": "Numer Kryterium ESG"
    },
    text_auto='.1%',
)

fig.update_layout(
    yaxis_tickformat=".0%",
    legend_title_text='Zbiór Danych',
    font=dict(family="Arial, sans-serif", size=12),
    template="plotly_white",
    uniformtext_minsize=8, 
    uniformtext_mode='hide',
    width=800,
    font_family="Times New Roman",
    font_color="black",
)

fig.update_traces(textposition='outside')
fig.show()

#### **1.6. Zapis podzielonych zbiorów do plików .jsonl.**

In [10]:
output_dir = "data/data_split"
os.makedirs(output_dir, exist_ok=True)

datasets_to_save = {
    "train": train_df,
    "validation": val_df,
    "test": test_df
}

for name, df_to_save in datasets_to_save.items():
    file_path = os.path.join(output_dir, f"{name}.jsonl")
    df_to_save.to_json(
        file_path,
        orient='records',
        lines=True,
        force_ascii=False,
        index=False
    )